In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)

## 1. Load Raw Data

In [ ]:
# Load the dataset
data_path = '../data/Coursera.csv'
df = pd.read_csv(data_path)

print(f"Loaded {len(df):,} courses")
print(f"Columns: {df.columns.tolist()}")

## 2. Clean Course Descriptions

In [ ]:
def clean_description(text):
    """
    Clean course description text:
    - Remove HTML tags
    - Remove special characters and weird encodings
    - Normalize whitespace
    - Remove URLs
    """
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Fix common encoding issues (ï¿½)
    text = text.replace('ï¿½', "'")
    text = text.replace('â€™', "'")
    text = text.replace('â€œ', '"')
    text = text.replace('â€', '"')
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# Test the function
sample_text = df['Course Description'].iloc[0]
print("Original:")
print(sample_text[:200])
print("\nCleaned:")
print(clean_description(sample_text)[:200])

In [ ]:
# Apply cleaning to all descriptions
print("Cleaning course descriptions...")
df['Course Description Clean'] = df['Course Description'].apply(clean_description)

# Show improvement
print(f"\nOriginal avg length: {df['Course Description'].str.len().mean():.0f} chars")
print(f"Cleaned avg length: {df['Course Description Clean'].str.len().mean():.0f} chars")
print(f"Empty descriptions: {(df['Course Description Clean'] == '').sum()}")

## 3. Clean Skills Data

In [ ]:
def clean_skills(skills_str):
    """
    Parse and clean skills string:
    - Split by multiple spaces (delimiter in dataset)
    - Remove duplicates
    - Remove empty/invalid entries
    - Standardize capitalization
    """
    if pd.isna(skills_str) or not isinstance(skills_str, str):
        return []
    
    # Split by multiple spaces
    skills = [s.strip() for s in skills_str.split('  ')]
    
    # Filter out empty and very short entries
    skills = [s for s in skills if s and len(s) > 1]
    
    # Remove duplicates (case-insensitive)
    seen = set()
    unique_skills = []
    for skill in skills:
        skill_lower = skill.lower()
        if skill_lower not in seen:
            seen.add(skill_lower)
            unique_skills.append(skill)
    
    return unique_skills

# Test the function
sample_skills = df['Skills'].iloc[0]
print("Original:")
print(sample_skills)
print("\nCleaned:")
print(clean_skills(sample_skills))

In [ ]:
# Apply skill cleaning
print("Cleaning skills data...")
df['Skills Clean'] = df['Skills'].apply(clean_skills)
df['Skills Count'] = df['Skills Clean'].apply(len)

print(f"\nSkill statistics:")
print(f"  Courses with skills: {(df['Skills Count'] > 0).sum():,}")
print(f"  Courses without skills: {(df['Skills Count'] == 0).sum():,}")
print(f"  Avg skills per course: {df['Skills Count'].mean():.1f}")
print(f"  Max skills per course: {df['Skills Count'].max()}")
print(f"  Min skills per course: {df['Skills Count'].min()}")

## 4. Handle Missing Values

In [ ]:
# Check for missing values in key columns
print("Missing values before cleaning:")
print(df[['Course Name', 'University', 'Course Rating', 'Difficulty Level']].isnull().sum())

# Fill missing ratings with 0.0
df['Course Rating'] = df['Course Rating'].fillna(0.0)

# Fill missing difficulty with 'Intermediate'
df['Difficulty Level'] = df['Difficulty Level'].fillna('Intermediate')

print("\nMissing values after cleaning:")
print(df[['Course Name', 'University', 'Course Rating', 'Difficulty Level']].isnull().sum())

## 5. Create Cleaned Dataset

In [ ]:
# Create final cleaned dataset
df_clean = df[[
    'Course Name',
    'University',
    'Difficulty Level',
    'Course Rating',
    'Course URL',
    'Course Description Clean',
    'Skills Clean'
]].copy()

# Rename columns
df_clean.columns = [
    'course_name',
    'university',
    'difficulty',
    'rating',
    'url',
    'description',
    'skills'
]

print(f"Cleaned dataset shape: {df_clean.shape}")
df_clean.head()

## 6. Data Quality Checks

In [10]:
print("Data Quality Report:")
print("=" * 60)

# Check for duplicate course names
duplicates = df_clean['course_name'].duplicated().sum()
print(f"Duplicate course names: {duplicates}")

# Check description lengths
desc_lengths = df_clean['description'].str.len()
print(f"\nDescription lengths:")
print(f"  Avg: {desc_lengths.mean():.0f} chars")
print(f"  Min: {desc_lengths.min()} chars")
print(f"  Max: {desc_lengths.max()} chars")
print(f"  Empty: {(desc_lengths == 0).sum()}")

# Convert rating to numeric for statistics
df_clean['rating'] = pd.to_numeric(df_clean['rating'], errors='coerce')

# Check rating distribution
print(f"\nRating distribution:")
print(f"  Mean: {df_clean['rating'].mean():.2f}")
print(f"  Median: {df_clean['rating'].median():.2f}")
print(f"  Std: {df_clean['rating'].std():.2f}")

# Check difficulty distribution
print(f"\nDifficulty distribution:")
print(df_clean['difficulty'].value_counts().to_string())


Data Quality Report:
Duplicate course names: 106

Description lengths:
  Avg: 1145 chars
  Min: 4 chars
  Max: 8547 chars
  Empty: 0

Rating distribution:
  Mean: 4.55
  Median: 4.60
  Std: 0.34

Difficulty distribution:
difficulty
Beginner          1444
Advanced          1005
Intermediate       837
Conversant         186
Not Calibrated      50


## 7. Save Cleaned Data

In [11]:
# Create output directory if it doesn't exist
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save cleaned data
output_path = output_dir / 'coursera_cleaned.csv'

# Convert skills list to string for CSV export
df_export = df_clean.copy()
df_export['skills'] = df_export['skills'].apply(lambda x: '  '.join(x) if x else '')

df_export.to_csv(output_path, index=False)
print(f"✓ Cleaned data saved to: {output_path}")
print(f"✓ Total courses saved: {len(df_export):,}")

# Also save as pickle to preserve list format
pickle_path = output_dir / 'coursera_cleaned.pkl'
df_clean.to_pickle(pickle_path)
print(f"✓ Cleaned data (with list format) saved to: {pickle_path}")

✓ Cleaned data saved to: ..\data\processed\coursera_cleaned.csv
✓ Total courses saved: 3,522
✓ Cleaned data (with list format) saved to: ..\data\processed\coursera_cleaned.pkl


## 8. Summary

In [12]:
print("\n" + "="*80)
print("DATA CLEANING SUMMARY")
print("="*80)
print(f"Input: {len(df):,} courses")
print(f"Output: {len(df_clean):,} courses")
print(f"\nCleaning operations performed:")
print("  ✓ Cleaned course descriptions (removed HTML, fixed encoding)")
print("  ✓ Parsed and deduplicated skills")
print("  ✓ Handled missing values")
print("  ✓ Standardized column names")
print(f"\nOutput files:")
print(f"  ✓ {output_path}")
print(f"  ✓ {pickle_path}")
print("="*80)


DATA CLEANING SUMMARY
Input: 3,522 courses
Output: 3,522 courses

Cleaning operations performed:
  ✓ Cleaned course descriptions (removed HTML, fixed encoding)
  ✓ Parsed and deduplicated skills
  ✓ Handled missing values
  ✓ Standardized column names

Output files:
  ✓ ..\data\processed\coursera_cleaned.csv
  ✓ ..\data\processed\coursera_cleaned.pkl
